In [3]:
from model.yolo import get_model

model = get_model('resnet152', (7,7), C=20)

In [6]:
import torch
model(torch.rand(1,3,224,224))

tensor([[[[-1.4936e-01,  4.5265e-02, -1.0676e-01,  ..., -2.0877e-01,
            1.5762e-01, -6.8534e-02],
          [ 5.0120e-03, -4.9954e-02,  1.9404e-01,  ...,  6.9962e-02,
           -2.4194e-01, -2.4866e-01],
          [ 3.0230e-01, -1.8303e-01,  3.3346e-02,  ...,  1.6087e-01,
            3.7520e-02,  2.4401e-01],
          ...,
          [-2.1483e-01, -5.1293e-01, -4.0860e-01,  ...,  5.0656e-02,
           -2.0325e-02,  2.5821e-01],
          [-1.5849e-01, -3.0949e-02, -1.6989e-01,  ...,  7.7901e-02,
            1.5059e-01,  3.0491e-01],
          [ 1.1941e-01,  3.2576e-02, -9.9799e-02,  ...,  3.6105e-01,
            1.6539e-02,  1.0862e-01]],

         [[ 4.4980e-01,  1.7316e-01, -8.5401e-02,  ..., -7.8344e-02,
            6.0846e-02, -2.3737e-01],
          [ 1.2521e-01, -8.5359e-02,  3.1233e-02,  ..., -2.6778e-01,
           -8.9496e-02, -7.0953e-02],
          [-1.9125e-01,  1.7632e-02,  1.0982e-02,  ...,  7.1079e-02,
           -1.2805e-01, -7.9721e-02],
          ...,
     